### Callback API Task

In [1]:
from keras.datasets import mnist

# tensorflow 라이브러리에 내장되어 있는 손글씨 데이터 세트 호출 후 데이터 세트 분리
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 형태 확인
print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

train dataset shape: (60000, 28, 28) (60000,)
test dataset shape: (10000, 28, 28) (10000,)


In [2]:
from tensorflow.keras.layers import Layer, Input, Flatten, Dense
from tensorflow.keras.models import Model

# 입력 데이터 크기 지정
INPUT_SIZE = 28

# 인공 신경망 생성 함수 선언
def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [3]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 전처리 함수 선언
def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

# 원-핫 인코딩 변환 함수 선언
def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

# 데이터 세트 분리 함수 선언
def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [4]:
# 데이터 세트 분리 함수를 통해 데이터 세트 분리
(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_labels, test_images, test_labels)

# 데이터 형태 확인
print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [5]:
# 현재 디렉토리 내 특정 폴더 확인
!dir callback_files

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 0457-852B

 C:\study\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28  오후 09:51    <DIR>          .
2024-05-28  오후 09:50    <DIR>          ..
               0개 파일                   0 바이트
               2개 디렉터리  1,822,810,292,224 바이트 남음


In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# weights 저장
mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

# 일정 기간 동안 성능이 개선되지 않을 시 학습률 동적으로 감소
rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

# 일정 기간 동안 성능이 개선되지 않을 시 학습 조기 중단
ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

# 훈련
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.8253 - loss: 0.6104 - val_acc: 0.9483 - val_loss: 0.1794 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9555 - loss: 0.1570 - val_acc: 0.9574 - val_loss: 0.1410 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9686 - loss: 0.1057 - val_acc: 0.9640 - val_loss: 0.1161 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.9743 - loss: 0.0829 - val_acc: 0.9643 - val_loss: 0.1089 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.9802 - loss: 0.0661 - val_acc: 0.9730 - val_loss: 0.0878 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.9853 - loss: 0.0492 - val_acc: 0.9682 - val_loss: 0.1031 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.9861 - loss: 0.0486 - val_acc: 0.9703 - val_loss: 0.0963 - learning_rate: 0.0010
Epoch 

In [7]:
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.9742 - loss: 0.0837


[0.0733608677983284, 0.9776999950408936]

In [8]:
# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 저장된 가중치 파일을 불러오기
model.load_weights('./callback_files/weights.016-0.0799-0.9979.weights.h5')

# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.9742 - loss: 0.0837


[0.0733608677983284, 0.9776999950408936]